WEB SCRAPING FROM AMAZON WITH MICROSOFT EDGE GETTING USEFUL INFORMATION AND PREPARING CSV FILE 

파이썬 라이브러리
Python Libraries

In [80]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import random
import time

In [81]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

프록시를 사용하여 아마존에 의해 블록되지 않도록 하기 위해
Using Proxies

In [82]:
PROXIES = [
'http://118.34.105.254:8080',
'http://115.144.99.223:11119',
'http://218.158.230.43:7890',
'http://121.132.95.7:80',
'http://115.144.123.219:10297',
'http://121.133.5.61:80',
'http://183.109.79.190:80',
'http://124.198.11.101:12425',
'http://106.244.154.91:8080',
'http://59.15.28.113:3128',
'http://121.126.200.123:11361',
'http://115.144.99.220:11116',
'http://115.144.1.222:12089'
]

random_proxy = random.choice(PROXIES)
seleniumwire_options = {
    'proxy': {
        'http': f'{random_proxy}',
        'https': f'{random_proxy}',
        'verify_ssl': False,
    },
}

검색어
Search Term and Selenium click if asking for cookies permission

In [83]:
search_term='Protein'
scrape_url=f'https://www.amazon.com/s?k={search_term}'
data=[]
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install(),seleniumwire_options=seleniumwire_options))



driver.get(scrape_url+"&page=1")

try:
    driver.find_element_by_css_selector("#sp-c-accpept").click()
except Exception as e:
    pass


마지막 페이지 수 찾기
Finding the last page nubmer

In [84]:
soup=BeautifulSoup(driver.page_source)
pages=soup.find("span",class_="s-pagination-strip").find_all("span")
last_page=int(pages[-1].text)
last_page

7

웹 스크래핑 (검색된 검색어 가격 제목 및 링크)
Web Scraping with Microsoft Edge

In [85]:
for i in range(1,last_page+1):
     items= soup.find_all("div",class_="a-section a-spacing-base")
    
     for item in items:
        item_price = item.find("span",class_="a-offscreen")
        if item_price==None:
            item_price='NA'
        else:
            item_price=item_price.text    
        item_title=item.find("a",class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal").text  
        item_url="https://www.amazon.com/"+item.find("a",class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")['href']
        data.append({"title":item_title,"price":item_price,"url":item_url,'page':i})
        #break
     driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install(),seleniumwire_options=seleniumwire_options))

     driver.get(scrape_url+"&page="+str(i+1))    
     try:
        driver.find_element_by_css_selector("#sp-c-accpept").click()
     except Exception as e:
         pass
     soup=BeautifulSoup(driver.page_source)

#item_price        
#item_title
#item_url

데이터 확인 

Checking our data

In [86]:
len(data)

485

In [91]:
data[300]

{'title': 'Nutricost Whey Protein Concentrate (Unflavored) 2LBS - Gluten Free & Non-GMO ',
 'price': '$29.95',
 'url': 'https://www.amazon.com//Nutricost-Whey-Protein-Concentrate-Unflavored/dp/B01KITQF0Q/ref=sr_1_260?keywords=Protein&qid=1687427476&sr=8-260',
 'page': 4}

In [92]:
df=pd.DataFrame(data)

In [93]:
df

,title,price,url,page
0,MuscleTech Burn iQ Smart Thermo Supplement Fue...,$59.99,https://www.amazon.com//sspa/click?ie=UTF8&spc...,1
1,"Ensure COMPLETE Nutrition Shake, 30g of Protei...",$69.78,https://www.amazon.com//sspa/click?ie=UTF8&spc...,1
2,Ascent 100% Whey Protein Powder - Post Workout...,$43.32,https://www.amazon.com//sspa/click?ie=UTF8&spc...,1
3,EHPlabs OxyWhey Whey Protein Powder Protein Sh...,$44.95,https://www.amazon.com//sspa/click?ie=UTF8&spc...,1
4,"Dymatize ISO100 Hydrolyzed Protein Powder, 100...",$33.39,https://www.amazon.com//Dymatize-Hydrolyzed-Ab...,1
...,...,...,...,...
480,Personalized Protein Powder 12.7 Oz.,$47.95,https://www.amazon.com//Personalized-Protein-P...,7
481,"JYM Supplement Science Iso JYM Watermelon, 90 ...",$35.22,https://www.amazon.com//JYM-Watermelon-Calorie...,7
482,PERFECT Sports Diesel 100% New Zealand Whey Is...,$64.99,https://www.amazon.com//PERFECT-SPORTS-Zealand...,7
483,"NOW Sports Nutrition, Certified Organic Whey P...",NA,https://www.amazon.com//Sports-Nutrition-Organ...,7


CSV 만들기
Creating a CSV

In [94]:
df.to_csv(f'./amazon-scraped-keyword{search_term}.csv',index=False)

중복된 값이 있는지 확인하기

Checking if there's any duplicate values


In [99]:
df.duplicated().sum()

0